### Неделя 5 Прогнозирование с помощью регрессии

In [1]:
%pylab inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import datetime
import statsmodels.api as sm
from itertools import product
from sklearn import cross_validation, linear_model, metrics
import scipy.cluster.hierarchy as sch

Populating the interactive namespace from numpy and matplotlib


C:\Users\JB\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\Users\JB\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


 Для каждой из шести задач прогнозирования $^​T+i∣T​,i=1,…,6$ сформируйте выборки. 
Откликом будет $yT+i​$​ при всевозможных значениях T, а признаки можно использовать следующие:

1. идентификатор географической зоны — категориальный
2. год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так
3. синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
4. сами значения прогнозов ARIMA 
5. количество поездок из рассматриваемого района в моменты времени $yT​,yT−1​,…,yT−K$ (параметр KKK можно подбирать; попробуйте начать, например, с 6)
6. количество поездок из рассматриваемого района в моменты времени $yT−24​,yT−48​,…,yT−24∗Kd​​$ (параметр KdK_dKd​ можно подбирать; попробуйте начать, например, с 2)
7. суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц

Будьте внимательны при создании признаков — все факторы должны быть рассчитаны без использования информации из будущего: при прогнозировании $^T+i∣T,i=1,…,6$ вы можете учитывать только значения yyy до момента времени TTT включительно.

In [2]:
# обучаться будем с 2014 по 04.2016
train_from = pd.to_datetime('2014-01-01 00:00:00')
train_to = pd.to_datetime('2016-04-30 17:00:00')

test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')

In [3]:
# подготовил предварительно данные по 102 ячейкам за январь 2014 - июнь 2016
data = pd.read_csv('data2014-2016_m6_102cell.csv', sep=';', header=0, index_col=['date_hour'], 
                  parse_dates=['date_hour'], dayfirst=True)

In [4]:
data.head(2)

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
date_hour,,,,,,,,,,,,,,,,,,,,,
2014-01-01 00:00:00,87,146,70,113,367,645,589,799,948,321,...,9,0,5,89,10,35,9,106,22,71
2014-01-01 01:00:00,92,184,93,153,539,604,490,635,667,225,...,24,0,3,22,2,5,0,87,0,44


In [5]:
data.tail(2)

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
date_hour,,,,,,,,,,,,,,,,,,,,,
2016-06-30 22:00:00,106,168,103,125,317,476,405,508,578,259,...,3,19,5,358,387,169,12,206,146,0
2016-06-30 23:00:00,85,130,86,113,256,428,483,531,631,210,...,9,0,7,323,110,147,38,173,119,0


In [6]:
data.shape

(21887, 102)

In [7]:
# сделаю копию выборки и буду с ней экспериментировать
dt_1 = data.copy()
dt_1.reset_index(inplace=True) # выведем date_hour из индекса в обычный столбец
dt_1.shape

(21887, 103)

In [8]:
dt_1.head(2)

,date_hour,1075,1076,1077,1125,1126,1127,1128,1129,1130,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
0,2014-01-01 00:00:00,87,146,70,113,367,645,589,799,948,...,9,0,5,89,10,35,9,106,22,71
1,2014-01-01 01:00:00,92,184,93,153,539,604,490,635,667,...,24,0,3,22,2,5,0,87,0,44


In [9]:
%%time
# разверну данные в длину по зонам
dt_long1 = pd.DataFrame(columns=['cell', 'date_hour','val'])
for i in xrange(1, len(dt_1.columns)):
    dt_temp = pd.DataFrame(columns=['cell', 'date_hour','val'])
    dt_temp['date_hour'] = dt_1.date_hour.values
    dt_temp['val'] = dt_1[dt_1.columns[i]].values
    dt_temp['cell'] = [dt_1.columns[i]] * dt_1.shape[0]
    
    dt_long1 = dt_long1.append(dt_temp)
    

Wall time: 8.69 s


In [10]:
dt_long1.reset_index(inplace=True)
dt_long1.drop(dt_long1.columns[0], inplace=True, axis=1)

In [27]:
#%%time
print dt_long1.shape
dt_long1.fillna(0,inplace=True)
dt_long1.head(3)

(2232474, 68)


,cell,date_hour,val,day,month,weekday,hour,year,sh_1,sh_2,...,sin_6,cos_6,sin_7,cos_7,target_1,target_2,target_3,target_4,target_5,target_6
0,1075,2014-01-01 00:00:00,87,1,1,2,0,2014,0.0,0.0,...,0.222521,0.974928,0.258819,0.965926,92.0,108.0,77.0,47.0,22.0,10.0
1,1075,2014-01-01 01:00:00,92,1,1,2,1,2014,5.0,0.0,...,0.433884,0.900969,0.500000,0.866025,108.0,77.0,47.0,22.0,10.0,18.0
2,1075,2014-01-01 02:00:00,108,1,1,2,2,2014,16.0,21.0,...,0.623490,0.781831,0.707107,0.707107,77.0,47.0,22.0,10.0,18.0,19.0


In [28]:
dt_long1.tail(2)

,cell,date_hour,val,day,month,weekday,hour,year,sh_1,sh_2,...,sin_6,cos_6,sin_7,cos_7,target_1,target_2,target_3,target_4,target_5,target_6
2232472,2168,2016-06-30 22:00:00,0,30,6,3,22,2016,-1.0,0.0,...,0.900969,0.433884,-0.965926,-2.588190e-01,0.0,0.0,0.0,0.0,0.0,0.0
2232473,2168,2016-06-30 23:00:00,0,30,6,3,23,2016,0.0,-1.0,...,0.974928,0.222521,-1.000000,-3.400451e-11,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
%%time
# Добавляем признаки дня, месяца, дня недели, часа и года.
dt_long1['day'] = dt_long1.date_hour.map(lambda x: x.day)
dt_long1['month'] = dt_long1.date_hour.map(lambda x: x.month)
dt_long1['weekday'] = dt_long1.date_hour.map(lambda x: x.weekday())
dt_long1['hour'] = dt_long1.date_hour.map(lambda x: x.hour)
dt_long1['year'] = dt_long1.date_hour.map(lambda x: x.year)

Wall time: 26 s


In [15]:
%%time
# Добавляем признаки числа поездок из этого региона за прошлые часы и дни
for i in xrange(1, 24):
    dt_long1['sh_' + str(i)] = dt_long1['val'] - dt_long1['val'].shift(i)

Wall time: 593 ms


In [17]:
%%time
for i in xrange(1, 15):
    dt_long1['sd_' + str(i)] = dt_long1['val'] - dt_long1['val'].shift(24*i)

Wall time: 375 ms


In [18]:
%%time
dt_long1['sum_p_12_h'] = [dt_long1['val'][i-12:i].sum() for i in xrange(dt_long1.shape[0])]
dt_long1['sum_p_24_h'] = [dt_long1['val'][i-24:i].sum() for i in xrange(dt_long1.shape[0])]
dt_long1['sum_p_w'] = [dt_long1['val'][i-24*7:i].sum() for i in xrange(dt_long1.shape[0])]

Wall time: 8min 54s


In [19]:
%%time
# Создадим регрессионные признаки для учёта сезонностей и трендов.
#Si=sin([1,…,T]∗2πi/168),Ci=cos([1,…,T]∗2πi/168),i=1,…,K.
# К возьму 7
itog_param = []
temp_param = []
K=7
arr_param = np.zeros((2232474,14), dtype=float)

#задублирую на все зоны
for j in range(K):
    temp_param = []
    for i in range(len(arr_param)):    
        arr_param[i,j*2] = sin((i+1)*2*np.pi*(j+1)/168.)
        arr_param[i,j*2+1] = cos((i+1)*2*np.pi*(j+1)/168.)
        
# добавляем синусы и косинусы в итоговый фрейм
for i in range(arr_param.shape[1]/2):
    dt_long1['sin_'+str(i+1)] = arr_param[:,i*2]
    dt_long1['cos_'+str(i+1)] = arr_param[:,i*2+1]

Wall time: 32.7 s


In [22]:
# Добавляем целевые значения для каждой из 6 моделей
for i in xrange(1, 7):
    dt_long1['target_' + str(i)] = dt_long1['val'].shift(-i)

In [25]:
len(X_train)

2082534

In [24]:
# подготовлю тест и трейн
X_train = dt_long1[(dt_long1.date_hour >= train_from) & (dt_long1.date_hour <= train_to)]
X_train.drop(['date_hour','target_1','target_2','target_3','target_4','target_5','target_6'],
             inplace=True, axis=1)
X_test = dt_long1[(dt_long1.date_hour >= test_from) & (dt_long1.date_hour <= test_to)]
X_test.drop(['date_hour','target_1','target_2','target_3','target_4','target_5','target_6'],
             inplace=True, axis=1)


C:\Users\JB\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\JB\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [52]:
%%time
# Create linear regression object для для 6-ти концов истории своя модель
# заодно посчитаю ошибку за май
models = dict()
Qmay = 0 # ошибка предсказания за май

for i in xrange(1,7):
    
    # определяю целевые значения, т.е. сдвинутые на 1 - 6 часов
    y_train = dt_long1[(dt_long1.date_hour >= train_from) & (dt_long1.date_hour <= train_to)]['target_'+str(i)].values
    y_test = dt_long1[(dt_long1.date_hour >= test_from) & (dt_long1.date_hour <= test_to)]['target_'+str(i)].values
    
    clf_LIN = linear_model.LinearRegression()
    clf_LIN.fit(X_train, y_train)
    pred_LIN = clf_LIN.predict(X_test)
    score_LIN = clf_LIN.score(X_test, y_test)
    
# занулим отриательные предсказания
    pred_LIN[pred_LIN < 0] = 0  # All low values set to 0
    
    models[i] = (i, clf_LIN)
    
    Qmay += sum(abs(y_test-pred_LIN))

    print 'точность модели linear regression для Т+' + str(i) + ' = ' + str(score_LIN)

точность модели linear regression для Т+1 = 0.971145365098
точность модели linear regression для Т+2 = 0.938406286643
точность модели linear regression для Т+3 = 0.916505151295
точность модели linear regression для Т+4 = 0.903575242355
точность модели linear regression для Т+5 = 0.897442676005
точность модели linear regression для Т+6 = 0.895283338759
Wall time: 1min 5s


точность падает чем дальше в будущее строится модель, хотелось бы понять почему

Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза

In [53]:
# считаем ошибку прогноза
1/(102*739*6.)*Qmay

33.25634368080204

ошибка стала меньше, на 4-й неделе была 42

 Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл 

In [73]:
# для предсказания июня обучаться будем с 2014 по 05.2016
train_from = pd.to_datetime('2014-01-01 00:00:00')
train_to = pd.to_datetime('2016-05-31 23:00:00')

test_from = pd.to_datetime('2016-05-31 23:00:00')
test_to = pd.to_datetime('2016-06-30 17:00:00')

X_train = dt_long1[(dt_long1.date_hour >= train_from) & (dt_long1.date_hour <= train_to)]
X_train.drop(['date_hour','target_1','target_2','target_3','target_4','target_5','target_6'],
             inplace=True, axis=1)
X_test = dt_long1[(dt_long1.date_hour >= test_from) & (dt_long1.date_hour <= test_to)]
X_test.drop(['date_hour','target_1','target_2','target_3','target_4','target_5','target_6'],
             inplace=True, axis=1)

# итоговый фрейм для кэгла
dt_itog = pd.DataFrame(dt_long1[(dt_long1.date_hour >= test_from) & 
                                (dt_long1.date_hour <= test_to)][['date_hour','cell']])

C:\Users\JB\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\JB\Anaconda2\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [75]:
%%time
# Create linear regression object для для 6-ти концов истории своя модель
# заодно посчитаю ошибку за июнь
models = dict()
Qjune = 0 # ошибка предсказания за май

for i in xrange(1,7):
    
    # определяю целевые значения, т.е. сдвинутые на 1 - 6 часов
    y_train = dt_long1[(dt_long1.date_hour >= train_from) & (dt_long1.date_hour <= train_to)]['target_'+str(i)].values
    y_test = dt_long1[(dt_long1.date_hour >= test_from) & (dt_long1.date_hour <= test_to)]['target_'+str(i)].values
    
    clf_LIN = linear_model.LinearRegression()
    clf_LIN.fit(X_train, y_train)
    pred_LIN = clf_LIN.predict(X_test)
    score_LIN = clf_LIN.score(X_test, y_test)
    
# занулим отриательные предсказания
    pred_LIN[pred_LIN < 0] = 0  # All low values set to 0
    
    models[i] = (i, clf_LIN)
    
    Qjune += sum(abs(y_test-pred_LIN))
    
    dt_itog[str(i)] = pred_LIN

    print 'точность модели linear regression для Т+' + str(i) + ' = ' + str(score_LIN)

точность модели linear regression для Т+1 = 0.971175310047
точность модели linear regression для Т+2 = 0.942189909525
точность модели linear regression для Т+3 = 0.923310005596
точность модели linear regression для Т+4 = 0.912912125571
точность модели linear regression для Т+5 = 0.907807002161
точность модели linear regression для Т+6 = 0.906099920836
Wall time: 1min 10s


In [76]:
# считаем ошибку прогноза
1/(102*739*6.)*Qjune

29.834049423253042

In [102]:
# функция по приведения даты к нкжному формату
def date_str(timed):
    if (timed).strftime('%H')=='00': 
        a = str((timed).strftime('%Y-%m-%d_'))+'0' 
    else: a = str((timed).strftime('%Y-%m-%d_'))+str((timed).strftime('%H')).lstrip("0")
    return a

In [77]:
dt_itog.head(2)

,date_hour,cell,1,2,3,4,5,6
21166,2016-05-31 23:00:00,1075,20.022723,17.930056,11.820684,5.557862,5.114939,7.472430
21167,2016-06-01 00:00:00,1075,25.525291,16.832089,9.975382,11.347308,18.415621,31.084236


In [104]:
%%time
# соберем теперь файл для кэгла
arr_pred_june = [] # итоговый массив
dt_Kaggle = pd.DataFrame()
for i in range(dt_itog.shape[0]):
    for j in xrange(1,7):
        arr_pred_june.append([str(dt_itog.cell.iloc[i]) + '_' + date_str(dt_itog.date_hour.iloc[i]) + '_' + str(j),
                            int(round(dt_itog[str(j)].iloc[i]))])

Wall time: 21.1 s


In [97]:
len(arr_pred_june)

437580

In [105]:
all_pd = pd.DataFrame(arr_pred_june)
all_pd.columns = ['id','y']

In [107]:
all_pd.head(2)

,id,y
0,1075_2016-05-31_23_1,20
1,1075_2016-05-31_23_2,18


In [108]:
all_pd.to_csv('itog5.csv', sep=',', index=False)

Score на kaggle стал лучше на 3% благодаря регрессии  https://yadi.sk/i/Rp8l-JG23Zouni

ссылка на решение https://www.kaggle.com/submissions/8069139/8069139.zip